# Demo of RESQML Attribute Conversion
Attributes covered are (Geoscience schema object names in brackets):
* Discrete (IntegerAttribute)
* Continuous (ContinuousAttribute)
* Categorical (CategoryAttribute)
* Points (VectorAttribute)

In [ ]:
from pprint import pprint

Import our conversion code

In [ ]:
from evo.data_converters.common.evo_client import EvoWorkspaceMetadata, create_evo_object_service_and_data_client
from evo.data_converters.resqml.importer._attribute_converters import (
    convert_categorical_property,
    convert_continuous_property,
    convert_discrete_property,
    convert_points_property,
    create_category_lookup_and_data,
)

Create a data_client object which we can use to save data

In [ ]:
_, data_client = create_evo_object_service_and_data_client(EvoWorkspaceMetadata(org_id="", workspace_id=""))

And import the target Geoscience Objects

In [ ]:
from evo_schemas.components import CategoryAttribute_V1_0_1 as CategoryAttribute
from evo_schemas.components import ContinuousAttribute_V1_0_1 as ContinuousAttribute
from evo_schemas.components import IntegerAttribute_V1_0_1 as IntegerAttribute
from evo_schemas.components import VectorAttribute_V1_0_0 as VectorAttribute

Create a RESQML Model and add a RegularGrid to it

In [ ]:
from os import path

import resqpy.grid as rqg
import resqpy.model as rqm
import resqpy.property as rqp

In [ ]:
# Create a Model and add a Grid to it
model_file = "demo_attribute_conversion.epc"
mymodel = rqm.new_model(model_file)

mygrid = rqg.RegularGrid(mymodel, extent_kji=(2, 3, 4), title="Test Grid")
mygrid.write_hdf5()
mygrid.create_xml(add_relationships=False, write_active=False, write_geometry=False, add_cell_length_properties=False)

Create some utility code to help with generating and checking the data

In [ ]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq


def get_data_from_parquet_file(pq_hash, data_client):
    return pq.read_table(path.join(data_client.cache_location, pq_hash)).to_pandas()


def check_data_is_valid(resqml_data, go_data):
    flattened_values = resqml_data.flatten()
    resqml_data_as_df = pd.DataFrame(flattened_values, columns=["data"])
    for resqml, go in zip(resqml_data_as_df, go_data):
        if resqml != go:
            return False
    return True


def check_lookup_table_is_valid(resqml_lookup_df, go_lookup_df):
    return resqml_lookup_df.size == go_lookup_df.size and np.all(resqml_lookup_df == go_lookup_df)


# Random data generator
rand = np.random.default_rng()

# RESQML Discrete Property

In [ ]:
discrete_data = rand.integers(1, 10, mygrid.extent_kji)
discrete_property = rqp.Property.from_array(
    mymodel,
    discrete_data,
    discrete=True,
    source_info="test data",
    property_kind="DiscreteProperty",
    indexable_element="cells",
    keyword="Discrete Property Test",
    support_uuid=mygrid.uuid,
    uom="m",
)

Convert the RESQML Discrete Property -> Geoscience Object IntegerAttribute

In [ ]:
property = discrete_property
title = property.citation_title
go = convert_discrete_property(property, data_client)
pprint(go)
if isinstance(go, IntegerAttribute):
    print(f"Property '{title}' was returned as an IntegerAttribute")
else:
    print("Boo!")

View the discrete data

In [ ]:
print(pq.read_table(path.join(data_client.cache_location, go.values.data)).to_pandas())

Check that the source data from RESQML matches the data in our Geoscience Object

In [ ]:
if check_data_is_valid(discrete_property.array_ref(), get_data_from_parquet_file(go.values.data, data_client)):
    print("Data is OK")
else:
    print("Data is INVALID")

# RESQML Continuous Property

In [ ]:
continuous_data = rand.random(mygrid.extent_kji).astype(np.float64)
continuous_property = rqp.Property.from_array(
    mymodel,
    continuous_data,
    dtype=np.float64,
    discrete=False,
    source_info="test data",
    property_kind="ContinuousProperty",
    indexable_element="cells",
    keyword="Continuous Property Test",
    support_uuid=mygrid.uuid,
    uom="m",
)

In [ ]:
property = continuous_property
title = property.citation_title
go = convert_continuous_property(property, data_client)
pprint(go)
if isinstance(go, ContinuousAttribute):
    print(f"Property '{title}' was returned as a ContinuousAttribute")
else:
    print("Boo!")

View continuous data

In [ ]:
print(pq.read_table(path.join(data_client.cache_location, go.values.data)).to_pandas())

Check that the source data from RESQML matches the data in our Geoscience Object

In [ ]:
if check_data_is_valid(continuous_property.array_ref(), get_data_from_parquet_file(go.values.data, data_client)):
    print("Data is OK")
else:
    print("Data is INVALID")

# RESQML Categorical Property

In [ ]:
# create a set of category labels in our model
string_lookup = rqp.StringLookup(mymodel)
string_lookup.set_string("0", "sandstone")
string_lookup.set_string("1", "shale")
string_lookup.set_string("2", "limestone")
string_lookup.create_xml()

# lookup table
lookup_as_dict = string_lookup.as_dict()
indices = list(lookup_as_dict.keys())
names = lookup_as_dict.values()
table_df = pd.DataFrame({"data": names, "index": indices})
table_df.set_index("index", inplace=True)
lookup_table_df, values_df = create_category_lookup_and_data(table_df)

# randomly assign a category to each cell
categorical_data = rand.integers(0, 3, size=mygrid.extent_kji)
categorical_property = rqp.Property.from_array(
    mymodel,
    categorical_data,
    discrete=True,
    source_info="test data",
    property_kind="CategoricalProperty",
    indexable_element="cells",
    keyword="Categorical Property Test",
    support_uuid=mygrid.uuid,
    string_lookup_uuid=string_lookup.uuid,
    uom="m",
)

In [ ]:
property = categorical_property
title = property.citation_title
go = convert_categorical_property(mymodel, property, data_client)
pprint(go)

In [ ]:
if isinstance(go, CategoryAttribute):
    print(f"Property '{title}' was returned as a CategoryAttribute")
else:
    print(f"ERROR {title} was NOT returned as a CategoryAttribute!")

View the resulting Geoscience Object lookup table, followed by the categorical data

In [ ]:
print(pq.read_table(path.join(data_client.cache_location, go.table.data)).to_pandas())
print(pq.read_table(path.join(data_client.cache_location, go.values.data)).to_pandas())

Check the data values are valid

In [ ]:
if check_data_is_valid(categorical_property.array_ref(), get_data_from_parquet_file(go.values.data, data_client)):
    print("Data is OK")
else:
    print("Data is INVALID")

Check the lookup table is ok

In [ ]:
if check_lookup_table_is_valid(lookup_table_df, get_data_from_parquet_file(go.table.data, data_client)):
    print("Lookup table is OK")
else:
    print("Lookup table is INVALID")
print(f"RESQML Lookup:\n{lookup_table_df}")
print(f"GO Lookup:\n{get_data_from_parquet_file(go.table.data, data_client)}")

# RESQML Points Property

In [ ]:
import resqpy.olio.vector_utilities as vec

if mygrid.property_collection is None:
    mygrid.property_collection = rqp.PropertyCollection(support=mygrid)
pc = mygrid.property_collection

# Define shape to be the grid plus x,y,z points
points_extent = tuple(list(mygrid.extent_kji) + [3])

# Create a static points property with some random stress data
stress = vec.unit_vectors(np.random.random(points_extent) + 0.1)
pc.add_cached_array_to_imported_list(
    cached_array=stress,
    source_info="random stress vectors",
    keyword="stress direction",
    uom="m",
    property_kind="length",
    indexable_element="cells",
    points=True,
)
pc.write_hdf5_for_imported_list()
pc.create_xml_for_imported_list_and_add_parts_to_model()
mymodel.store_epc()

points_part = pc.singleton(citation_title="stress direction", points=True)
stress_uuid = pc.uuid_for_part(points_part)
points_property = rqp.Property(mymodel, uuid=stress_uuid)

In [ ]:
property = points_property
title = property.citation_title
go = convert_points_property(property, data_client)
pprint(go)

In [ ]:
if isinstance(go, VectorAttribute):
    print(f"Property '{title}' was returned as a VectorAttribute")
else:
    print(f"ERROR {title} was NOT returned as a VectorAttribute!")

View the resulting Geoscience Object lookup table, followed by the categorical data

In [ ]:
print(pq.read_table(path.join(data_client.cache_location, go.values.data)).to_pandas())

Check the data is Ok

In [ ]:
prop_data = points_property.array_ref().reshape(-1, 3)
go_data = get_data_from_parquet_file(go.values.data, data_client).to_numpy()
if go_data.all() == prop_data.all():
    print("Data is OK")
else:
    print("Data is INVALID")